In [4]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset
from torchvision import models, transforms
from sklearn.metrics import cohen_kappa_score
from PIL import Image
import os
import tqdm

In [5]:

#GeM layer(contribution)
from torch.nn.parameter import Parameter

def gem(x, p=3, eps=1e-6):
    return F.avg_pool2d(x.clamp(min=eps).pow(p), (x.size(-2), x.size(-1))).pow(1./p)

class GeM(nn.Module):
    def __init__(self, p=3, eps=1e-6):
        super(GeM, self).__init__()
        self.p = Parameter(torch.ones(1)*p)
        self.eps = eps

    def forward(self, x):
        return gem(x, p=self.p, eps=self.eps)

    def __repr__(self):
        return self.__class__.__name__ + '(' + 'p=' + '{:.4f}'.format(self.p.data.tolist()[0]) + ', ' + 'eps=' + str(self.eps) + ')'


In [6]:

# Load your train_csv and test_csv
train_csv = pd.read_csv("/kaggle/input/aptos2019-blindness-detection/train.csv")
test_csv = pd.read_csv("/kaggle/input/aptos2019-blindness-detection/test.csv")

# Define data paths
trainpath ="/kaggle/input/aptos2019-blindness-detection/train_images"
testpath = "/kaggle/input/aptos2019-blindness-detection/test_images"

# Define transformation(contribution)
apply_transform = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ColorJitter(brightness=20, contrast=0.2, saturation=20, hue=10),
    transforms.RandomAffine(degrees=180, scale=(1-0.2, 1+0.2), shear=0.2),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
])


In [7]:
# Create tensors for images and labels
TrainImages = torch.FloatTensor(len(train_csv), 3, 128, 128)
TrainLabels = torch.LongTensor(len(train_csv))
TestImages = torch.FloatTensor(len(test_csv), 3, 128, 128)


In [8]:
# Load and preprocess training images
for i in tqdm.tqdm(range(len(train_csv))):
    img_name, label = train_csv.values[i]
    img_path = os.path.join(trainpath, img_name + '.png')
    image = Image.open(img_path)
    image = apply_transform(image)
    TrainImages[i] = image
    TrainLabels[i] = label


100%|██████████| 3662/3662 [10:17<00:00,  5.93it/s]


In [9]:
# Load and preprocess test images
for i in tqdm.tqdm(range(len(test_csv))):
    img_name = test_csv['id_code'][i]
    img_path = os.path.join(testpath, img_name + '.png')
    image = Image.open(img_path)
    image = apply_transform(image)
    TestImages[i] = image

100%|██████████| 1928/1928 [01:59<00:00, 16.19it/s]


In [10]:
# Split into train and validation sets(contribution)
train_num = int(0.8 * len(TrainImages))
val_num = len(TrainImages) - train_num

Train_Images = TrainImages[:train_num]
ValImages = TrainImages[train_num:]
Train_Labels = TrainLabels[:train_num]
ValLabels = TrainLabels[train_num:]


In [11]:
# Define datasets and dataloaders
trainDataset = TensorDataset(Train_Images, Train_Labels)
valDataset = TensorDataset(ValImages, ValLabels)
testDataset = TensorDataset(TestImages)

trainDataLoader = DataLoader(trainDataset, batch_size=16, shuffle=True, num_workers=2, pin_memory=True)
valDataLoader = DataLoader(valDataset, batch_size=16, shuffle=True, num_workers=2, pin_memory=True)
testDataLoader = DataLoader(testDataset, batch_size=16, shuffle=False, num_workers=2, pin_memory=True)


In [12]:
#(contribution)
class CustomModel(nn.Module):
    def __init__(self):
        super(CustomModel, self).__init__()
        # Convolutional layers
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1)
        self.pool = GeM()

        # Additional convolutional layers
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)

        # Fully connected layers
        self.fc1 = nn.Linear(0, 128)  # The input size will be set dynamically during the forward pass
        self.fc2 = nn.Linear(128, 5)

    def forward(self, x):
        # Convolutional layers
        x = self.conv1(x)
        x = self.pool(x)

        # Additional convolutional layers
        x = self.conv2(x)
        x = self.pool(x)

        # Flatten the feature map
        x = x.view(x.size(0), -1)

        # Set the input size of fc1 dynamically
        if self.fc1.in_features == 0:
            self.fc1.in_features = x.size(1)
            self.fc1 = nn.Linear(x.size(1), 128)

        # Fully connected layers
        x = F.relu(self.fc1(x))
        x = self.fc2(x)

        return x

    
model = CustomModel()

/opt/conda/lib/python3.10/site-packages/torch/nn/init.py:405: UserWarning: Initializing zero-element tensors is a no-op
  warnings.warn("Initializing zero-element tensors is a no-op")


In [13]:
# Move the model to the GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

CustomModel(
  (conv1): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool): GeM(p=3.0000, eps=1e-06)
  (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (fc1): Linear(in_features=0, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=5, bias=True)
)

In [14]:
# Loss, optimizer, scheduler
criterion = nn.SmoothL1Loss()
optimizer = optim.Adam(model.parameters(), lr=0.0000015)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1)


In [15]:
# Training loop
iterations = 25
trainLoss = []

for epoch in range(iterations):
    runningLoss = 0
    model.train()
    
    for data in tqdm.tqdm(trainDataLoader):
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.long().to(device)
        inputs = inputs / 255

        outputs = model(inputs)

        # Ensure that the size of labels matches the size of the output
        labels = labels.unsqueeze(1)  # Add a singleton dimension at position 1

        loss = criterion(F.log_softmax(outputs, dim=1), labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        runningLoss += loss.item()

    avgTrainLoss = runningLoss / (train_num / 32)
    trainLoss.append(avgTrainLoss)

    # Validation logic here

    scheduler.step()


  0%|          | 0/184 [00:00<?, ?it/s]/opt/conda/lib/python3.10/site-packages/torch/nn/modules/loss.py:928: UserWarning: Using a target size (torch.Size([16, 1])) that is different to the input size (torch.Size([16, 5])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.smooth_l1_loss(input, target, reduction=self.reduction, beta=self.beta)
 99%|█████████▉| 183/184 [01:10<00:00,  2.55it/s]/opt/conda/lib/python3.10/site-packages/torch/nn/modules/loss.py:928: UserWarning: Using a target size (torch.Size([1, 1])) that is different to the input size (torch.Size([1, 5])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.smooth_l1_loss(input, target, reduction=self.reduction, beta=self.beta)
100%|██████████| 184/184 [01:12<00:00,  2.54it/s]


In [17]:
# Testing loop
model.eval()
TestLabels = torch.LongTensor(len(test_csv), 1928)
all_predictions = []

for i in range(10):
    test_pred = np.array([])

    for data in tqdm.tqdm(testDataLoader):
        inputs = data[0]
        inputs = inputs / 255
        outputs = model(inputs)

        _, predicted = torch.max(outputs, 1)
        test_pred = np.append(test_pred, predicted.cpu().numpy())

    all_predictions.append(test_pred)

    TestLabels[i] = torch.from_numpy(test_pred).unsqueeze(0)

TestLabels = (torch.sum(TestLabels, dim=0)) / 10
TestLabels = TestLabels.numpy().astype(int)

# Flatten all predictions from different iterations
all_predictions_flat = np.concatenate(all_predictions)

# Compute accuracy
true_labels = TestLabels  # Provide the true labels for the test dataset
accuracy = np.sum(np.array_equal(all_predictions_flat, true_labels)) / len(true_labels)

print(f"Accuracy: {accuracy * 100:.2f}%")

# Display predictions
print("Predictions:", all_predictions_flat)


100%|██████████| 121/121 [00:14<00:00,  8.16it/s]

Accuracy: 82.63%
Predictions: [1. 1. 1. ... 1. 1. 1.]
